In [2]:
from math import sqrt
import torch
from torch_geometric.data import Data
from random import randint
from sys import float_info

In [3]:
instances = {}
for k in range(0, 5000):
    nodes = {}
    for i in range(0, 50):
        lat_i = randint(0, 100)
        lon_i = randint(0, 100)
        node_i = (lat_i, lon_i)
        lat_j = randint(0, 100)
        lon_j = randint(0, 100)
        node_j = (lat_j, lon_j)
        nodes[i + 1] = node_i
        nodes[i + 51] = node_j

    dist = {}
    pairs = {}
    for i in range(1, 101):
        for j in range(1, 101):
            if i != j:
                dist[i,j] = sqrt( (nodes[i][0] - nodes[j][0])**2 + (nodes[i][1] - nodes[j][1])**2 )
            else:
                dist[i,j] = float_info.max
    for i in range(1, 101):
        for j in range(1, 101):
            if i not in pairs:
                pairs[i] = j
            if i != j:
                if dist[i,j] < dist[i,pairs[i]]:
                    pairs[i] = j

    nodes[0] = (0,0)
    for i in range(1,101):
        dist[0,i] = sqrt( (nodes[0][0] - nodes[i][0])**2 + (nodes[0][1] - nodes[i][1])**2 )
        dist[i,0] = dist[0,i]
    y = [[0 for _ in range(101)] for _ in range(101)]
    for i in range(101):
        if i > 0:
            y[i][pairs[i]] = 1
                
    instances[k] = {"nodes": nodes, "dist": dist, "y": y}

In [4]:
instances[0]["y"]

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [3]:

import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.loader import DataLoader

In [4]:
complete_graph_list = []
for i in range(101):
    for j in range(101):
        if i != j:
            complete_graph_list.append([i,j])
edge_index = torch.tensor(complete_graph_list, dtype=torch.double).t().contiguous()
n_edges = len(complete_graph_list)

In [6]:
data_list = []
for instance_name in instances:
    y = torch.tensor(instances[instance_name]["y"], dtype=torch.double)
    x = torch.tensor([instances[instance_name]["nodes"][i] for i in range(0, 101)], dtype=torch.double)
    attr = [[i] for i in range(n_edges)]
    cnt = -1
    for i in range(101):
        for j in range(101):
            if i != j:
                cnt += 1
                attr[cnt].append(instances[instance_name]["dist"][i,j])
    attr = torch.tensor(attr, dtype=torch.double)
    pos = []
    for i in range(101):
        pos.append(instances[instance_name]["nodes"][i])
    pos = torch.tensor(pos, dtype=torch.double)
    # ## filtering by TW, strict
    # complete_graph_list = []
    # for i in range(101):
    #     for j in range(101):
    #         if i!=j:
    #             try:
    #                 if instance_dict[instance_name][i][5] + instance_dict[instance_name][i][6] + loc_dict[i][j] < instance_dict[instance_name][i][5]:
    #                     complete_graph_list.append([i,j])
    #             except:
    #                 continue
    # edge_index = torch.tensor(complete_graph_list, dtype=torch.double).t().contiguous()
    ## end filtering
    data_list.append(Data(x=x, y=y, edge_index=edge_index, pos=pos, edge_attr=attr))

In [86]:
# data_source = Instances(data_list)
dataloader = DataLoader(data_list[0:100], batch_size=1)
data_test = DataLoader(data_list[100:120], batch_size=1)
# datatorch = data_source.to_conv_nets(start=428, end=1458, batch_size=10)
# torchtest = data_source.to_conv_nets(start=1458, end=488, batch_size=10)

In [67]:
import torch
from torch.nn import Sequential as Seq, Linear, ReLU, Softmax
from torch_geometric.nn import MessagePassing

class EdgeConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super().__init__(aggr='max') #  "Max" aggregation.
        self.mlp = Seq(Linear(2 * in_channels, out_channels),
                       ReLU(),
                       Linear(out_channels, out_channels),
                       Softmax(dim=1))

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        # x_i has shape [E, in_channels]
        # x_j has shape [E, in_channels]

        tmp = torch.cat([x_i, x_j - x_i], dim=1)  # tmp has shape [E, 2 * in_channels]
        return self.mlp(tmp)

In [68]:
from torch_geometric.nn import knn_graph

class DynamicEdgeConv(EdgeConv):
    def __init__(self, in_channels, out_channels, k=6):
        super().__init__(in_channels, out_channels)
        self.k = k
        self.double()

    def forward(self, x, batch=None):
        edge_index = knn_graph(x, self.k, batch, loop=False, flow=self.flow)
        return super().forward(x, edge_index)

In [69]:
def myCustomLoss(my_outputs, objective_matrix):
    #specifying the batch size
    my_batch_size = my_outputs.size()[0]
    #selecting the values that correspond to labels
    diff = torch.abs(my_outputs - objective_matrix)
    return torch.sum(diff*diff)/(101*100)

In [70]:
def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    loss_all = 0
    for data in dataloader:
        data = data.to(device)
        optimizer.zero_grad()
        # print(data.x, data.edge_index, data.batch)
        output = model(data.x)
        # print(output, data.y)
        # print(output)
        loss = myCustomLoss(output, data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
        test_size = 101*101*1 #extract batch size
    return loss_all / test_size

def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x)
        pred = torch.tensor([[1 if j>0.5 else 0 for j in i] for i in output.tolist()], dtype=torch.double)
        correct += pred.eq(data.y).sum().item()
        test_size = 101*101*1 #extract batch size
    return correct / test_size / len(loader)

In [17]:
from copy import deepcopy

In [71]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DynamicEdgeConv(2, 101).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model_dict = {}

for epoch in range(1, 31):
    loss = train(epoch)
    train_acc = test(dataloader)
    test_acc = test(data_test)
    model_dict[epoch] = {"model": deepcopy(model), "loss": loss, "trainAcc": train_acc, "testAcc": test_acc}
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

ValueError: Expected 'edge_index' to be of integer type (got 'torch.float64')

In [36]:
best_model = model_dict[max(model_dict, key=lambda k: model_dict[k]["testAcc"])]["model"]
best_model.eval()
data_show = data_list[430]
pred = best_model(data_show.x).tolist()

In [37]:
max_val = -2
min_val = 2
vals = []
for i in pred:
    vals += i
    if max(i) > max_val:
        max_val = max(i)
    if min(i) < min_val:
        min_val = min(i)
from statistics import mean, stdev
avg_val = mean(vals)
sd_val = stdev(vals)
threshold = (max_val + min_val)/2

In [47]:
row_dif_dict = {}
for k in range(0, 1000):
    row_dif = 0
    for i in range(len(pred)):
        # print(sum([1 if j > avg_val + 3.9 * sd_val else 0 for j in pred[i]]), int(sum(data_show.y[i])), abs(sum([1 if j > avg_val + 3.9 * sd_val else 0 for j in pred[i]]) - int(sum(data_show.y[i]))))
        row_dif += abs(sum([1 if j > avg_val + (0 + k * 0.1) * sd_val else 0 for j in pred[i]]) - int(sum(data_show.y[i])))
    row_dif_dict[k] = row_dif
best_threshold = min(row_dif_dict, key=lambda k: row_dif_dict[k])
# for i in range(len(pred)):
#     print(sum([1 if j > avg_val + (3.9 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]]), int(sum(data_show.y[i])), abs(sum([1 if j > avg_val + (3.9 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]]) - int(sum(data_show.y[i]))))
# print(best_threshold, row_dif_dict[best_threshold])
for i in range(len(pred)):
    print([1 if j > avg_val + (0 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]])
cnt = 0
for i in range(len(pred)):
    cnt += sum([1 if j > avg_val + (0 + best_threshold * 0.1) * sd_val else 0 for j in pred[i]])
    if cnt > 0:
        print(i)
print(cnt)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [50]:
for i in range(len(pred)):
    print(data_show.y[i].tolist())
print(sum(sum(y)))
print(data_show.y[49].tolist())

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [82]:
from torch_geometric.nn import GraphConv, global_add_pool
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        num_features = 2
        dim = 101*101

        self.conv1 = GraphConv(num_features, dim)
        self.conv2 = GraphConv(dim, dim)

        self.lin1 = Linear(dim, dim)

        self.double()

    def forward(self, x, edge_index, batch, edge_weight=None):
        # x, edge_index = data.x, data.edge_index

        # x = self.conv1(x, edge_index)
        # x = F.relu(x)
        # x = F.sigmoid(self.conv2(x, edge_index))
        # x = F.dropout(x, training=self.training)
        # x = self.conv2(x, edge_index)

        x = self.conv1(x, edge_index, edge_weight).relu()
        x = self.conv2(x, edge_index, edge_weight).relu()
        x = global_add_pool(x, batch)
        x = self.lin1(x).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        # x = self.lin2(x)
        x = torch.reshape(x, (101, 101))
        return x

In [89]:
def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    loss_all = 0
    for data in dataloader:
        data = data.to(device)
        optimizer.zero_grad()
        # print(data.x, data.edge_index, data.batch)
        output = model(data.x, data.edge_index.long(), data.batch)
        # print(output, data.y)
        # print(output)
        loss = myCustomLoss(output, data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
        test_size = 101*100*1 #extract batch size
    return loss_all / test_size

def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.edge_index.long(), data.batch)
        pred = torch.tensor([[1 if j>0.5 else 0 for j in i] for i in output.tolist()], dtype=torch.double)
        correct += pred.eq(data.y).sum().item()
        test_size = 101*101*1 #extract batch size
    return correct / test_size / len(loader)

In [90]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model_dict = {}

for epoch in range(1, 31):
    loss = train(epoch)
    train_acc = test(dataloader)
    test_acc = test(data_test)
    model_dict[epoch] = {"model": deepcopy(model), "loss": loss, "trainAcc": train_acc, "testAcc": test_acc}
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: 109581186955.2469, Train Acc: 0.9931, Test Acc: 0.9932
Epoch: 002, Loss: 304529.5171, Train Acc: 0.9932, Test Acc: 0.9932
Epoch: 003, Loss: 1499.2428, Train Acc: 0.9932, Test Acc: 0.9932


KeyboardInterrupt: 

New simple model

The plan is to simply consider the problem of a point as a separation problem between neighbors as clustering per distance. Let's consider a problem with d = (i = (|V| - 1)) + (l = 2) + (t = 2) dimensions. Dimensions i represent the remainder nodes distances, l represent the current position and t the time windows. Each node has this properties. This simple net will have at least 3 neurons with no depth.

In [5]:
import numpy as np
import keras as ks
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt

In [23]:
X = np.matrix([[instances[k]["dist"][i,j] if i != j else sqrt(100*100.0 + 100*100.0) for j in range(101)] for i in range(101) for k in range(1000)])
y = np.matrix([[1 if instances[k]["y"][i][j] > 0.5 else 0 for j in range(101)] for k in range(1000) for i in range(101)])

In [25]:
print(sum(instances[0]["y"][1]))
print(sum(y[1].tolist()[0]))
print(instances[0]["y"][1][14])
print(y[1].tolist()[0])

1
1
1
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [26]:
loss_fun = ks.losses.BinaryFocalCrossentropy()
model1 = Sequential()
model1.add(Dense(202, input_dim=101, activation='relu'))
model1.add(Dense(101, activation='sigmoid'))
# model2 = Sequential()
# model2.add(Dense(5, input_dim=101, activation='relu'))
# model2.add(Dense(1, activation='sigmoid'))
# model3 = Sequential()
# model3.add(Dense(20, input_dim=101, activation='relu'))
# model3.add(Dense(1, activation='sigmoid'))
# compile the models. Here we need to chose an optimiser. This one is called adam, it is used to
# determine how the training is performed. We do not care in this lecture how this is done.
opt = ks.optimizers.Adam(learning_rate=0.02)
model1.compile(loss=loss_fun, optimizer=opt, metrics=['accuracy'])
# model2.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
# model3.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
#fit the models on the dataset
model1.fit(X, y, epochs=30, batch_size=5, verbose = True)
# model2.fit(X, y, epochs=30, batch_size=5, verbose = False)
# model3.fit(X, y, epochs=30, batch_size=5, verbose = False)
# evaluate the keras model
print('Model 1:')
_, accuracy1 = model1.evaluate(X, y)
# print('Model 2:')
# _, accuracy2 = model2.evaluate(X, y)
# print('Model 3:')
# _, accuracy3 = model3.evaluate(X, y)

Epoch 1/30


c:\Users\jotape\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20200/20200 ━━━━━━━━━━━━━━━━━━━━ 14s 674us/step - accuracy: 0.0101 - loss: 0.0626
Epoch 2/30
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 20s 1ms/step - accuracy: 0.0094 - loss: 0.0172
Epoch 3/30
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 18s 903us/step - accuracy: 0.0111 - loss: 0.0172
Epoch 4/30
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 19s 920us/step - accuracy: 0.0105 - loss: 0.0172
Epoch 5/30
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 19s 954us/step - accuracy: 0.0095 - loss: 0.0172
Epoch 6/30
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 18s 906us/step - accuracy: 0.0102 - loss: 0.0172
Epoch 7/30
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 19s 915us/step - accuracy: 0.0100 - loss: 0.0172
Epoch 8/30
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 19s 951us/step - accuracy: 0.0093 - loss: 0.0172
Epoch 9/30
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 19s 925us/step - accuracy: 0.0096 - loss: 0.0172
Epoch 10/30
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 19s 948us/step - accuracy: 0.0092 - loss: 0.0172
Epoch 11/30
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 19s 955us/step - accuracy: 0.0096 - l

In [27]:
for i in range(101):
    op = [instances[1]["y"][i][j] for j in range(101)]
    print(i, sum(op), op)

0 0 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1 1 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2 1 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3 1 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [28]:
Z1 = model1.predict(np.matrix([instances[1]["dist"][1,j] if 1 != j else sqrt(100*100.0 + 100*100.0) for j in range(101)]))
# Z2 = model2.predict(np.c_[xx.ravel(), yy.ravel()])
# Z2 = Z2.reshape(xx.shape)
# Z3 = model3.predict(np.c_[xx.ravel(), yy.ravel()])
# Z3 = Z3.reshape(xx.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


In [29]:
print(Z1)
print(sum(sum(Z1)))

[[0.0004636  0.1559699  0.14335416 0.16537502 0.13699532 0.18546706
  0.15914968 0.13623002 0.17849146 0.15327127 0.10835808 0.15444154
  0.15246178 0.14017324 0.1523073  0.15369795 0.15136583 0.10452051
  0.2010394  0.16847779 0.12431862 0.14099012 0.14823768 0.20421444
  0.17488497 0.13805285 0.16422431 0.1248764  0.13973199 0.1309487
  0.15856494 0.16603626 0.13724701 0.13784201 0.14403546 0.13663472
  0.19482039 0.13039406 0.15996666 0.13588639 0.169056   0.13095038
  0.14058593 0.13518591 0.14505151 0.12443595 0.16295972 0.1540658
  0.13523653 0.1609689  0.13172956 0.13867827 0.14252095 0.1458379
  0.14133203 0.14206211 0.1750446  0.14771229 0.14198083 0.15966856
  0.14315094 0.19757378 0.17028202 0.17881708 0.15382896 0.17326878
  0.15533097 0.16323183 0.14739864 0.15828732 0.15376584 0.13919304
  0.1831731  0.12012734 0.15588646 0.15816142 0.12966993 0.14724664
  0.18580605 0.16935799 0.14665313 0.1592567  0.15345679 0.14810085
  0.14896096 0.14619163 0.12255137 0.16554141 0.148

In [36]:
print(np.matrix([float(instances[1]["y"][1][j]) for j in range(101)]))
print(sum([(instances[1]["y"][1][j]) for j in range(101)]))
print(instances[1]["y"][1][54], print(Z1.tolist()[0][54]))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]]
1
0.14133203029632568
1 None


In [31]:
print(sum(sum((Z1-np.matrix([float(instances[1]["y"][1][j]) for j in range(101)])[0].tolist()))))

14.119866843189811


In [37]:
loss_fun = ks.losses.BinaryFocalCrossentropy()
model1_1 = Sequential()
model1_1.add(Dense(202, input_dim=101, activation='linear'))
model1_1.add(Dense(101, activation='sigmoid'))
# model2 = Sequential()
# model2.add(Dense(5, input_dim=101, activation='relu'))
# model2.add(Dense(1, activation='sigmoid'))
# model3 = Sequential()
# model3.add(Dense(20, input_dim=101, activation='relu'))
# model3.add(Dense(1, activation='sigmoid'))
# compile the models. Here we need to chose an optimiser. This one is called adam, it is used to
# determine how the training is performed. We do not care in this lecture how this is done.
opt = ks.optimizers.Adam(learning_rate=0.02)
model1_1.compile(loss=loss_fun, optimizer=opt, metrics=['accuracy'])
# model2.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
# model3.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
#fit the models on the dataset
model1_1.fit(X, y, epochs=30, batch_size=5, verbose = True)
# model2.fit(X, y, epochs=30, batch_size=5, verbose = False)
# model3.fit(X, y, epochs=30, batch_size=5, verbose = False)
# evaluate the keras model
print('Model 1:')
_, accuracy1 = model1_1.evaluate(X, y)
# print('Model 2:')
# _, accuracy2 = model2.evaluate(X, y)
# print('Model 3:')
# _, accuracy3 = model3.evaluate(X, y)

Epoch 1/30


c:\Users\jotape\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20200/20200 ━━━━━━━━━━━━━━━━━━━━ 14s 673us/step - accuracy: 0.0099 - loss: 11.4286
Epoch 2/30
 7353/20200 ━━━━━━━━━━━━━━━━━━━━ 12s 994us/step - accuracy: 0.0104 - loss: 11.3211

KeyboardInterrupt: 

In [23]:
Z1 = model1_1.predict(np.matrix([instances[1]["dist"][1,j] if 1 != j else sqrt(100*100.0 + 100*100.0) for j in range(101)]))
# Z2 = model2.predict(np.c_[xx.ravel(), yy.ravel()])
# Z2 = Z2.reshape(xx.shape)
# Z3 = model3.predict(np.c_[xx.ravel(), yy.ravel()])
# Z3 = Z3.reshape(xx.shape)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [24]:
print(Z1)
print(sum(sum(Z1)))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]]
0.0


In [25]:
print(np.matrix([float(instances[1]["y"][1][j]) for j in range(101)]))
print(sum([(instances[1]["y"][1][j]) for j in range(101)]))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]]
2


In [38]:
from keras.optimizers import SGD
model1_2 = Sequential()
model1_2.add(Dense(202, input_dim=101, activation='relu', kernel_initializer='he_uniform'))
model1_2.add(Dense(202, input_dim=101, activation='linear'))
model1_2.add(Dense(101, input_dim=101, activation='linear'))
model1_2.add(Dense(101, activation='sigmoid'))
# model2 = Sequential()
# model2.add(Dense(5, input_dim=101, activation='relu'))
# model2.add(Dense(1, activation='sigmoid'))
# model3 = Sequential()
# model3.add(Dense(20, input_dim=101, activation='relu'))
# model3.add(Dense(1, activation='sigmoid'))
# compile the models. Here we need to chose an optimiser. This one is called adam, it is used to
# determine how the training is performed. We do not care in this lecture how this is done.
# opt = SGD(learning_rate=0.01, momentum=0.9)
opt = ks.optimizers.Adam()
model1_2.compile(loss='binary_focal_crossentropy', optimizer=opt, metrics=['accuracy'])
# model2.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
# model3.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])
#fit the models on the dataset
model1_2.fit(X, y, epochs=2, batch_size=5, verbose = True)
# model2.fit(X, y, epochs=30, batch_size=5, verbose = False)
# model3.fit(X, y, epochs=30, batch_size=5, verbose = False)
# evaluate the keras model
print('Model 1:')
_, accuracy1 = model1_2.evaluate(X, y)
# print('Model 2:')
# _, accuracy2 = model2.evaluate(X, y)
# print('Model 3:')
# _, accuracy3 = model3.evaluate(X, y)

Epoch 1/2
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 27s 1ms/step - accuracy: 0.0104 - loss: 0.0937
Epoch 2/2
20200/20200 ━━━━━━━━━━━━━━━━━━━━ 25s 1ms/step - accuracy: 0.0103 - loss: 0.0172
Model 1:
3157/3157 ━━━━━━━━━━━━━━━━━━━━ 3s 946us/step - accuracy: 0.0101 - loss: 0.0171


In [39]:
Z1 = model1_2.predict(np.matrix([instances[1]["dist"][1,j] if 1 != j else sqrt(100*100.0 + 100*100.0) for j in range(101)]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [40]:
print(Z1)
print(sum(sum(Z1)))

[[0.00306269 0.14397271 0.16418889 0.16890487 0.15206394 0.1458237
  0.15138115 0.1468649  0.1485856  0.14704347 0.13873418 0.15283997
  0.15898676 0.15278138 0.13427763 0.15019664 0.12964237 0.16258562
  0.15527074 0.16024078 0.14668085 0.13837795 0.15999517 0.16409256
  0.15215458 0.15548074 0.13825043 0.1409442  0.15453283 0.14152814
  0.17397183 0.16056266 0.16029322 0.13744913 0.13975306 0.15213749
  0.16046087 0.1401422  0.16663986 0.13287565 0.14790276 0.14235969
  0.14535575 0.13933806 0.1466801  0.16591397 0.15411355 0.16261391
  0.14178145 0.17368962 0.14059761 0.14156052 0.14544888 0.14208722
  0.15225399 0.1509302  0.1369633  0.12296626 0.14560314 0.17264143
  0.17278993 0.13423346 0.14935054 0.14447963 0.13261448 0.14628254
  0.1509145  0.15996547 0.15016888 0.14808175 0.13705997 0.13925408
  0.14702344 0.13826478 0.15136485 0.14147724 0.13825153 0.14864525
  0.13133207 0.13598439 0.14177537 0.12478901 0.13912341 0.16034855
  0.14473695 0.13966851 0.12908325 0.14803562 0.1

In [41]:
print(np.matrix([float(instances[1]["y"][1][j]) for j in range(101)]))
print(sum([(instances[1]["y"][1][j]) for j in range(101)]))

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]]
1


In [4]:
import numpy as np

In [42]:
graph_data = np.array([[[instances[k]["nodes"][i][0], instances[k]["nodes"][i][1]] for i in range(101)] for _ in range(101*100) for k in range(100)])
print("next stage")
node_pairs_i = []
node_pairs_j = []
objective_vals = []
for k in range(100):
    for i in range(101):
        for j in range(101):
            if i != j:
                node_pairs_i.append([instances[k]["nodes"][i][0], instances[k]["nodes"][i][1]])
                node_pairs_j.append([instances[k]["nodes"][j][0], instances[k]["nodes"][j][1]])
                objective_vals.append(instances[k]["y"][i][j])
node_pairs_i = np.matrix(node_pairs_i)
node_pairs_j = np.matrix(node_pairs_j)
objective_vals = np.array(objective_vals)

next stage


In [43]:
from keras.models import Model
from keras.layers import concatenate, Input, Dense, Flatten
# define two sets of inputs
inputA = Input(shape=(101,2))
inputB = Input(shape=(2,))
inputC = Input(shape=(2,))
# the first branch operates on the first input
x_graph = Dense(101*101, activation="linear")(inputA)
x_graph = Dense(101, activation="relu")(x_graph)
x_graph = Dense(2, activation="linear")(x_graph)
x_graph = Flatten()(x_graph)
x_graph = Model(inputs=inputA, outputs=x_graph)
# the second branch opreates on the second input
x_node_i = Dense(2, activation="linear")(inputB)
x_node_i = Model(inputs=inputB, outputs=x_node_i)
# the third branch opreates on the third input
x_node_j = Dense(2, activation="linear")(inputC)
x_node_j = Model(inputs=inputC, outputs=x_node_j)
# combine the output of the two branches
combined = concatenate([x_graph.output, x_node_i.output, x_node_j.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
z = Dense(101, activation="linear")(combined)
z = Dense(2, activation="relu")(z)
z = Dense(2, activation="linear")(z)
z = Dense(1, activation="sigmoid")(z)
# our model will accept the inputs of the two branches and
# then output a single value
model1_3 = Model(inputs=[x_graph.output, x_node_i.output, x_node_j.output], outputs=z)

In [44]:
from keras.optimizers import SGD

In [45]:
opt = SGD(learning_rate=0.01, momentum=0.9)
model1_3.compile(loss='binary_focal_crossentropy', optimizer=opt, metrics=['accuracy'])
model1_3.fit(x=[Flatten()(graph_data), node_pairs_i, node_pairs_j], y=objective_vals, epochs=2, batch_size=5, verbose = True)
print('Model 1_3:')
_, accuracy1 = model1_3.evaluate([Flatten()(graph_data), node_pairs_i, node_pairs_j], objective_vals)

Epoch 1/2
202000/202000 ━━━━━━━━━━━━━━━━━━━━ 164s 812us/step - accuracy: 0.9896 - loss: 0.0462
Epoch 2/2
202000/202000 ━━━━━━━━━━━━━━━━━━━━ 177s 876us/step - accuracy: 0.9901 - loss: 0.0173
Model 1_3:
31563/31563 ━━━━━━━━━━━━━━━━━━━━ 24s 748us/step - accuracy: 0.9901 - loss: 0.0173


In [46]:
cnt = 0
for i in range(len(objective_vals)):
    if objective_vals[i] == 1:
        print(i)
        cnt += 1
    if cnt > 10:
        break

113
234
307
491
572
645
766
803
967
1096
1144


In [57]:
graph_ref = []
for i in range(len(graph_data[233].tolist())):
    graph_ref.append(graph_data[233][i].tolist()[0])
    graph_ref.append(graph_data[233][i].tolist()[1])
graph_ref = np.array([graph_ref])

In [58]:
Z1 = model1_3.predict(x=[graph_ref, node_pairs_i[233], node_pairs_j[233]])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


In [59]:
print(Z1)

[[0.14328632]]


In [55]:
node_pairs_i[234]

matrix([[32, 13]])

In [56]:
node_pairs_j[234]

matrix([[34, 16]])

In [67]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

In [116]:


class KNNSimilarityLayer(Layer):
    def __init__(self, train_data, k=3, **kwargs):
        self.train_data = train_data
        self.k = k
        super(KNNSimilarityLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        super(KNNSimilarityLayer, self).build(input_shape)

    def call(self, inputs):
        # Expand dimensions to calculate distances
        inputs_expanded = tf.reshape(tf.expand_dims(inputs, 1), (101,2))
        t1 = tf.cast(tf.reshape(inputs_expanded, (1,101,2)), tf.float16)
        t2 = tf.cast(tf.reshape(inputs_expanded, (101,1,2)), tf.float16)
        
        # Calculate Euclidean distances
        distances = tf.norm(t1-t2, ord='euclidean', axis=2,)
        
        # Get the indices of the k nearest neighbors
        _, indices = tf.nn.top_k(-distances, k=self.k)

        # Create row indices (repeated for each column index in idx_tensor)
        row_indices = tf.repeat(tf.range(indices.shape[0]), indices.shape[1])

        # Flatten the column indices
        col_indices = tf.reshape(indices, [-1])

        # Combine row and column indices to form the full index tensor
        indices = tf.stack([row_indices, col_indices], axis=1)

        # Create updates tensor (all ones)
        updates = tf.ones(indices.shape[0], dtype=tf.int32)

        # Define the shape of the output tensor
        output_shape = [101, 101]

        # Scatter the updates into the output tensor
        prediction = tf.expand_dims(tf.scatter_nd(indices, updates, output_shape), 0)
        
        # # Gather the labels of the k nearest neighbors
        # knn_labels = tf.gather(self.train_labels, indices)
        
        # # Aggregate the labels (mode)
        # knn_labels = tf.cast(knn_labels, tf.float32)
        # prediction = tf.reduce_mean(knn_labels, axis=1)
        
        return prediction

In [60]:
from keras.layers import Reshape

In [68]:
inputs_expanded = tf.reshape(tf.expand_dims(unique_graph_data[0], 1), (101,2))
train_data_expanded = tf.expand_dims(unique_output_data[0], 0)[0]

In [69]:
train_data_expanded
print(tf.math.reduce_sum(train_data_expanded, 0))

tf.Tensor(
[0 0 1 1 0 0 2 2 2 1 2 1 2 2 2 0 0 2 2 0 1 3 1 1 2 1 1 0 1 1 1 1 1 1 0 1 0
 0 2 1 1 2 0 0 1 3 1 3 1 1 1 0 1 0 2 0 1 0 1 1 1 1 1 2 1 0 0 1 2 1 0 1 1 2
 2 0 1 1 0 2 0 1 2 0 1 1 1 1 1 0 1 3 1 1 0 0 1 1 1 1 0], shape=(101,), dtype=int32)


In [104]:
idx_tensor = tf.constant(np.array([[1 , 2], [0, 3]]))
# Create row indices (repeated for each column index in idx_tensor)
row_indices = tf.repeat(tf.range(idx_tensor.shape[0]), idx_tensor.shape[1])

# Flatten the column indices
col_indices = tf.reshape(idx_tensor, [-1])

# Combine row and column indices to form the full index tensor
indices = tf.stack([row_indices, col_indices], axis=1)

# Create updates tensor (all ones)
updates = tf.ones(indices.shape[0], dtype=tf.int32)

# Define the shape of the output tensor
output_shape = [2, 4]

# Scatter the updates into the output tensor
output_tensor = tf.scatter_nd(indices, updates, output_shape)

print(output_tensor)

tf.Tensor(
[[0 1 1 0]
 [1 0 0 1]], shape=(2, 4), dtype=int32)


In [70]:
t1 = tf.cast(tf.reshape(inputs_expanded, (1,101,2)), tf.float16)
t2 = tf.cast(tf.reshape(inputs_expanded, (101,1,2)), tf.float16)
result = tf.norm(t1-t2, ord='euclidean', axis=2,)

In [81]:
# print(result)
_, indices = tf.nn.top_k(-result, k=3)
print(indices)

tf.Tensor(
[[  0   1  14]
 [  1   0  14]
 [  2  35   4]
 [  3   8  23]
 [  4  92  99]
 [  5  73  62]
 [  6  46  49]
 [  7  67  43]
 [  8   3  23]
 [  9  68  22]
 [ 10  97  75]
 [ 11  45  95]
 [ 12  58  85]
 [ 13  72  78]
 [ 14  91  70]
 [ 15  47  55]
 [ 16  74  84]
 [ 17  82  19]
 [ 18  50  56]
 [ 19  17  82]
 [ 20  81  74]
 [ 21  38  57]
 [ 22  68   9]
 [ 23   8   3]
 [ 24  33  83]
 [ 25  52  31]
 [ 26  69  95]
 [ 27  56   7]
 [ 28  85  74]
 [ 29  64  77]
 [ 30  98  39]
 [ 31  90  25]
 [ 32  59  71]
 [ 33  24  83]
 [ 34  21  38]
 [ 35   2  64]
 [ 36  11  19]
 [ 37  38  21]
 [ 38  21  57]
 [ 39  62  30]
 [ 40  86  52]
 [ 41  60  89]
 [ 42   9  68]
 [ 43   7  60]
 [ 44  54  78]
 [ 45  73  95]
 [ 46   6 100]
 [ 47  77  55]
 [ 48  61  82]
 [ 49   6 100]
 [ 50  18  56]
 [ 51  23   8]
 [ 52  25  31]
 [ 53  82  17]
 [ 54  44  63]
 [ 55  47  77]
 [ 56  18  27]
 [ 57  21  38]
 [ 58  12  85]
 [ 59  32  93]
 [ 60  41  43]
 [ 61  48  82]
 [ 62  39  34]
 [ 63  76  54]
 [ 64  29  77]
 [ 65  63  76]

In [88]:
prediction = tf.Variable(tf.zeros((101,101)))
for i in range(101):
    for j in indices[i]:
        prediction[i][j].assign(1)
print(prediction)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

In [90]:
unique_graph_data = np.array([[[instances[k]["nodes"][i][0], instances[k]["nodes"][i][1]] for i in range(101)] for k in range(100)])

In [91]:
unique_output_data = []
for k in range(100):
    unique_output_data.append([])
    for l in y[101*k:101*(k+1)]:
        unique_output_data[k].append(l.tolist()[0])
unique_output_data = np.array(unique_output_data)

In [117]:
knn_layer = KNNSimilarityLayer( np.float32(unique_graph_data), k=3)


In [62]:
from keras.layers import Reshape

In [118]:
inputs = Input(shape=(101,2))
knn_output = knn_layer(inputs)
dense_output = Dense(32, activation='relu')(knn_output)
dense_output = Dense(16, activation='relu')(dense_output)
outputs = Dense(101, activation='sigmoid')(dense_output)
# outputs = Reshape((1, 101,101)) (outputs)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='binary_focal_crossentropy', metrics=['accuracy'])

In [121]:
tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [122]:
model.fit(np.float32(unique_graph_data), np.float32(unique_output_data), epochs=20, batch_size=1)

Epoch 1/20


c:\Users\jotape\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


InvalidArgumentError: {{function_node __wrapped__Sub_device_/job:localhost/replica:0/task:0/device:CPU:0}} Incompatible shapes: [101,32] vs. [0] [Op:Sub]